In [ ]:
!pip install embedchain


In [ ]:
!pip install docx2txt

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install dropbox


In [ ]:
!pip install pdf2image pytesseract

In [ ]:
!apt-get install poppler-utils

In [ ]:
!apt-get install tesseract-ocr
!apt-get install liptesseract-dev

In [14]:
import dropbox
import os
from pdf2image import convert_from_path
import pytesseract

from embedchain import App
# Replace this with your HF token
os.environ["HUGGINGFACE_ACCESS_TOKEN"] = "hfxxx"
os.environ["DROPBOX_ACCESS_TOKEN"] = "slxxx"
config = {
  'llm': {
    'provider': 'huggingface',
    'config': {
      'model': 'mistralai/Mistral-7B-Instruct-v0.2',
      'top_p': 0.5
    }
  },
  'embedder': {
    'provider': 'huggingface',
    'config': {
      'model': 'sentence-transformers/all-mpnet-base-v2'
    }
  }
}
# Dropbox API access token
ACCESS_TOKEN = 'slxxx'

# Create a Dropbox client
dbx = dropbox.Dropbox(ACCESS_TOKEN)

# Dropbox folder path
dropbox_folder_path = '/test'

# Local folder to save downloaded PDFs
local_folder_path = './downloaded_pdfs'
os.makedirs(local_folder_path, exist_ok=True)

# List and download PDFs



In [ ]:
def list_and_download_pdfs(dbx, folder_path):
    pdf_files = []
    result = dbx.files_list_folder(folder_path)
    while result.entries:
        for entry in result.entries:
            if isinstance(entry, dropbox.files.FileMetadata) and entry.name.endswith('.pdf'):

                local_file_path = os.path.join(local_folder_path, entry.name)
                if not os.path.exists(local_file_path):
                    pdf_files.append(entry.name)
                with open(local_file_path, "wb") as f:
                    metadata, res = dbx.files_download(entry.path_lower)
                    f.write(res.content)
        if result.has_more:
            result = dbx.files_list_folder_continue(result.cursor)
        else:
            break
    return pdf_files

pdf_files = list_and_download_pdfs(dbx, dropbox_folder_path)
print(f"Downloaded PDFs: {pdf_files}")

# Convert PDFs to text using OCR
def pdf_to_text(pdf_path):
    pages = convert_from_path(pdf_path, 300)
    texts = [pytesseract.image_to_string(page) for page in pages]
    return "\n".join(texts)

pdf_texts = []
for pdf_file in pdf_files:
    local_pdf_path = os.path.join(local_folder_path, pdf_file)
    text = pdf_to_text(local_pdf_path)
    pdf_texts.append((pdf_file, text))

app = App.from_config(config=config)
# Add extracted texts to EmbedChain
for pdf_file, text in pdf_texts:
    app.add(text, data_type='text')
app.add("/general", data_type="dropbox")
# Query the embedded texts

In [ ]:

query = "What are the course objectives for TEC 301"
response = app.query(query)
print(response)
